## Many-shot settings
Given many shot data, we are able to fine-tune the detector, which is equalevent to class-incremental learning.

In each stage, the model is fine-tuned on only the data in current stage

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
import sys
sys.path.append('..')

load_incremental_topic can be used by formatting the order

---
```python
# two stages. first stage includes 4 classes (with human) and the second stage incude 2 classes
order = [['gpt35', 'Mixtral','Moonshot',], ['Llama3','gpt-4omini']]
```
---
```python
# fives stages. first stage includes 2 classes (with human) and each remaining stage includes 1 class
order = [['Moonshot'],['Mixtral'],['gpt35'],['Llama3'],['gpt-4omini']]
```

In [4]:
order = [['Moonshot'],['Mixtral'],['gpt35'],['Llama3'],['gpt-4omini']]
order = [['gpt35', 'Mixtral','Moonshot','Llama3'],['gpt-4omini']]
from mgtbench.loading import load_incremental_topic, load_incremental

data = load_incremental_topic(order, "Social_sciences")
# data = load_incremental(order, "Art")

In [5]:
set(data['train'][0]['label']), set(data['train'][1]['label'])

({0, 1, 2, 3, 4}, {5})

In [6]:
nclass = len(set(data['train'][0]['label']))
nclass

5

In [7]:
from mgtbench import AutoDetector, AutoExperiment
method = 'incremental'
model= '/data1/models/roberta-base'
nclass = len(set(data['train'][0]['label']))

### Model-based Incremental Tenchniques
We implement LwF, iCaRL, BiC for adapatation, which are controlled by parameters:

`lwf_reg` is a distillation-based regularization parameter for LwF

`cache_size` is for iCaRL. Based on the class_mean, select the top-k closest samples in each class for cache

`bic` is for BiC layer. It calibrates the output logits for each class

Additionally, `factor` is decayed lr. For example, learning rate is initialized to lr in stage1, and rate is initialized to lr/factor in stage2.

In [8]:
metric = AutoDetector.from_detector_name(method, model_name_or_path=model,
                                        num_labels=nclass, lwf_reg = reg,cache_size=size,bic=bic)
experiment = AutoExperiment.from_experiment_name('incremental',detector=[metric])
config = {'need_finetune': True,
        'need_save': False,
        'epochs': 2,
        'lr': 1e-6,
        'batch_size':64,
        'save_path': '/data1/lyl/mgtout-1/',
        'eval':True,
        'lr_factor': factor
        }
# cfg = SupervisedConfig()
# cfg.update(config)
experiment.load_data(data)
res1 = experiment.launch(**config)


NameError: name 'reg' is not defined

In [ ]:
from mgtbench import AutoDetector, AutoExperiment
method = 'incremental'
model= '/data1/models/roberta-base'
nclass = len(set(data['train'][0]['label']))

scoring_model_name_or_path = '/data_sda/zhiyuan/models/gpt-neo-2.7B'
reference_model_name_or_path = '/data_sda/zhiyuan/models/gpt-j-6B'
detector = AutoDetector.from_detector_name('fast-detectGPT', 
                                            scoring_model_name_or_path=scoring_model_name_or_path,
                                            reference_model_name_or_path= reference_model_name_or_path
                                            )
experiment = AutoExperiment.from_experiment_name('incremental_threshold',detector=[detector],cache_size=0)
experiment.load_data(data)
experiment.launch()